# Conversation Knowledge Graph Memory

This type of memory uses a knowledge graph to recreate memory.

Let's first walk through how to use the utilities

In [1]:
from langchain.memory import ConversationKGMemory
from langchain.llms import OpenAI

In [2]:
llm = OpenAI(temperature=0)
memory = ConversationKGMemory(llm=llm)
memory.save_context({"input": "对张三说你好"}, {"ouput": "谁是张三"})
memory.save_context({"input": "张三是一个朋友"}, {"ouput": "好的"})

In [3]:
memory.load_memory_variables({"input": '谁是张三'})

{'history': 'On 张三: 张三 是 一个朋友.'}

We can also get the history as a list of messages (this is useful if you are using this with a chat model).

In [13]:
memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context({"input": "谢霆锋是张柏芝的什么？"}, {"ouput": "前夫"})
memory.save_context({"input": "张柏芝是陈冠希的什么？"}, {"ouput": "情人"})

In [17]:
memory.load_memory_variables({"input": '张柏芝的情人是谁？'})

{'history': [SystemMessage(content='On 谢霆锋: 谢霆锋 is 前夫) of (张柏芝.', additional_kwargs={}),
  SystemMessage(content='On 张柏芝: 张柏芝 是 陈冠希的情人.', additional_kwargs={})]}

We can also more modularly get current entities from a new message (will use previous messages as context.)

In [18]:
memory.get_current_entities("陈冠希的情人是谁？")

['陈冠希', '张柏芝', '谢霆锋']

We can also more modularly get knowledge triplets from a new message (will use previous messages as context.)

In [19]:
memory.get_knowledge_triplets("陈冠希的情人是谁？")

[KnowledgeTriple(subject='Chen Guanxi', predicate='has', object_='Zhang Baozhi'),
 KnowledgeTriple(subject='Zhang Baozhi', predicate='is', object_="Chen Guanxi's lover")]

## Using in a chain
Let's now use this in a chain!

In [29]:
llm = OpenAI(temperature=0)
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

template = """
以下是人类和AI之间友好的对话。
AI很健谈，并从其上下文中提供了很多具体的细节。
如果 AI 不知道问题的答案，它会诚实地说出来。
AI 仅使用“相关信息”部分中包含的信息，不会产生幻觉。

相关信息：

{history}

对话：
人类：{input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template
)
conversation_with_kg = ConversationChain(
    llm=llm, 
    verbose=True, 
    prompt=prompt,
    memory=ConversationKGMemory(llm=llm,k=2)
)

In [30]:
conversation_with_kg.predict(input="张柏芝的情人是陈冠希")



> Entering new ConversationChain chain...
Prompt after formatting:

以下是人类和AI之间友好的对话。
AI很健谈，并从其上下文中提供了很多具体的细节。
如果 AI 不知道问题的答案，它会诚实地说出来。
AI 仅使用“相关信息”部分中包含的信息，不会产生幻觉。

相关信息：



对话：
人类：张柏芝的情人是陈冠希
AI:

> Finished chain.


' 是的，据报道，张柏芝和陈冠希曾于2008年经历过一段恋情。'

In [31]:
conversation_with_kg.predict(input="谢霆锋是张柏芝老公")



> Entering new ConversationChain chain...
Prompt after formatting:

以下是人类和AI之间友好的对话。
AI很健谈，并从其上下文中提供了很多具体的细节。
如果 AI 不知道问题的答案，它会诚实地说出来。
AI 仅使用“相关信息”部分中包含的信息，不会产生幻觉。

相关信息：

On 张柏芝: 张柏芝 的情人是 陈冠希.

对话：
人类：谢霆锋是张柏芝老公
AI:

> Finished chain.


' 不，谢霆锋不是张柏芝的老公。据我所知，张柏芝的情人是陈冠希。'

In [32]:
conversation_with_kg.predict(input="谢霆锋和陈冠希是什么关系？")



> Entering new ConversationChain chain...
Prompt after formatting:

以下是人类和AI之间友好的对话。
AI很健谈，并从其上下文中提供了很多具体的细节。
如果 AI 不知道问题的答案，它会诚实地说出来。
AI 仅使用“相关信息”部分中包含的信息，不会产生幻觉。

相关信息：

On 张柏芝: 张柏芝 的情人 陈冠希.
On 谢霆锋: 谢霆锋 不是 张柏芝的老公.

对话：
人类：谢霆锋和陈冠希是什么关系？
AI:

> Finished chain.


' 谢霆锋不是张柏芝的老公，但他们曾经是情侣。'

In [34]:
conversation_with_kg.memory.get_knowledge_triplets("陈冠希的情人是谁？")

[KnowledgeTriple(subject='Chen Guanxi', predicate='has a partner', object_='Xie Tingfeng')]